In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.5.0'

In [ ]:
!git clone https://github.com/CartagenaMinas/Melanomas.git

Cloning into 'Melanomas'...
remote: Enumerating objects: 43603, done.
remote: Total 43603 (delta 0), reused 0 (delta 0), pack-reused 43603
Receiving objects: 100% (43603/43603), 219.76 MiB | 34.70 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (44116/44116), done.


In [ ]:
import os
os.listdir("Melanomas")

['Keras_baseline.ipynb', 'data', 'exploracion.ipynb', '.git']

In [ ]:
PATH="Melanomas/data"
os.listdir(PATH)

['jpeg224',
 'test.csv',
 'sample_submission.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv',
 'subset.csv']

In [ ]:
import pandas as pd
train=pd.read_csv(f"{PATH}/subset.csv")
val=pd.read_csv(f"{PATH}/val_split.csv")
train.shape,val.shape

((2220, 8), (10932, 8))

In [ ]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [ ]:
BATCH_SIZE=64
AUTO=tf.data.experimental.AUTOTUNE

def decode(name,label):
  img=tf.io.read_file(name)#lo lee en byts
  img=tf.image.decode_jpeg(img,channels=3)#lo carga como imagen
  img=tf.cast(img,tf.float32)#/255#cambia la presicion
  return img,label

def load_ds(df):
  options=tf.data.Options()
  options.experimental_deterministic=False#Para que lo cargue en orden aleatorio
  imgs, labels=df["image_name"].values,df["target"].values#Creo una lista 
  imgs=[f"{PATH}/jpeg224/train/{name}.jpg" for name in imgs]#Creo una lista con mis rutas
  ds=tf.data.Dataset.from_tensor_slices((imgs,labels))
  ds=ds.with_options(options)
  ds=ds.map(decode,num_parallel_calls=AUTO)#uso todos los procesadores
  ds=ds.cache()#Guarda la imagen en memoria, lo puedo usar si tengo mucha ram
  ds=ds.shuffle(2048)
  ds=ds.batch(BATCH_SIZE)
  ds=ds.prefetch(buffer_size=AUTO) #Mientras la gpu calcula la cpu carga imagenes
  return ds



In [ ]:
train_ds=load_ds(train)
val_ds=load_ds(val)

In [ ]:
IMAGE_SIZE= (224,224,3)

encoder=ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights="imagenet"
)
encoder.trainable=False #No aplicar los gradientes calculados

inputs=keras.Input(shape=IMAGE_SIZE)
x=keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x=encoder(x,training=False)#No calcula los gradientes
x=keras.layers.GlobalAveragePooling2D()(x)
outputs=keras.layers.Dense(1,activation="sigmoid")(x)
model=keras.Model(inputs,outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [ ]:
filepath="./checkpoints/checkpoint"
cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor="val_auc",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="max"
)

In [ ]:
model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    validation_steps=10,
    callbacks=[cb])

Epoch 1/10
35/35 [==============================] - 11s 322ms/step - loss: 0.0522 - auc: 0.9456 - val_loss: 0.1110 - val_auc: 0.7779

Epoch 00001: val_auc improved from -inf to 0.77791, saving model to ./checkpoints/checkpoint
Epoch 2/10
35/35 [==============================] - 11s 321ms/step - loss: 0.0495 - auc: 0.9536 - val_loss: 0.0652 - val_auc: 0.8359

Epoch 00002: val_auc improved from 0.77791 to 0.83587, saving model to ./checkpoints/checkpoint
Epoch 3/10
35/35 [==============================] - 11s 323ms/step - loss: 0.0480 - auc: 0.9601 - val_loss: 0.1042 - val_auc: 0.7695

Epoch 00003: val_auc did not improve from 0.83587
Epoch 4/10
35/35 [==============================] - 11s 325ms/step - loss: 0.0462 - auc: 0.9615 - val_loss: 0.1174 - val_auc: 0.7263

Epoch 00004: val_auc did not improve from 0.83587
Epoch 5/10
35/35 [==============================] - 11s 321ms/step - loss: 0.0448 - auc: 0.9638 - val_loss: 0.1102 - val_auc: 0.6717

Epoch 00005: val_auc did not improve from

In [ ]:
#Cargar los Pesos
model.load_weights(filepath)
model.save("model.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
test=pd.read_csv(f"{PATH}/test.csv")
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [ ]:
BATCH_SIZE=256

def decode_test(name):
  img=tf.io.read_file(name)#lo lee en byts
  img=tf.image.decode_jpeg(img,channels=3)#lo carga como imagen
  img=tf.cast(img,tf.float32)#/255#cambia la presicion
  return img

def load_test_ds(df):
  imgs=df["image_name"].values#Creo una lista 
  imgs=[f"{PATH}/jpeg224/test/{name}.jpg" for name in imgs]#Creo una lista con mis rutas
  ds=tf.data.Dataset.from_tensor_slices(imgs)
  ds=ds.map(decode_test)
  ds=ds.batch(BATCH_SIZE)
  return ds

In [ ]:
test_ds=load_test_ds(test)

In [ ]:
preds=model.predict(test_ds)

In [ ]:
preds

array([[0.00096433],
       [0.00272754],
       [0.0054366 ],
       ...,
       [0.01788147],
       [0.00041774],
       [0.0610103 ]], dtype=float32)

In [ ]:
submission=pd.DataFrame({
    "image_name":test["image_name"].values,"target":preds.ravel()
})
submission.head()

,image_name,target
0,ISIC_0052060,0.000964
1,ISIC_0052349,0.002728
2,ISIC_0058510,0.005437
3,ISIC_0073313,0.000975
4,ISIC_0073502,0.000621


In [ ]:
submission.to_csv("sumission1.csv",index=False)